# User Interface for the Property Assessement Database.


This user interface allows users to filter through database and return relevant data, below is also an interface for (authorized) editors to add or drop rows in the database. 

Website consulted: https://www.geeksforgeeks.org/how-to-connect-python-with-sql-database/

In [12]:
import mysql.connector
import ipywidgets as widgets
from IPython.display import display

# Connect to MySQL database
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="password",
        database="property_data"
    )

In [2]:
# Retrieve tables
def get_tables():
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES;")
    tables = [table[0] for table in cursor.fetchall()]
    cursor.close()
    connection.close()
    return tables

In [ ]:
# Retrieve columns
def get_columns(table_name):
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute(f"DESCRIBE {table_name};")
    columns = [column[0] for column in cursor.fetchall()]
    cursor.close()
    connection.close()
    return columns

## This is a function that allows users to search, filter, and retrieve data from the table.

In [3]:
# Description of each table for the user interface
def get_table_descriptions():
    return {
        "amenities" : "Provides a list of amenities for each property, such as heat, ac, parking, etc.\n",
        "bldg_composite" : "Provides information about building structure class, roof, exterior, etc.\n",
        "bldg_condition" : "Provide information about building condition (interior/exterior)\n",
        "bldg_info" : "Provides information about building units (residential, commercial, etc)\n",
        "mail" : "Provides information of a mailing address.\n",
        "property" : "Provides information on property such as owner, street address, etc.\n",
        "property_sf" : "Provides information about property space (land square footage, gross area, etc).\n",
        "property_type" : "Provides information about number of buildings, land usage, etc.\n ",
        "rooms" : "Provides information about number of bedrooms, bathrooms, total rooms, etc.\n",
        "style" : "Provides information about bedroom, bathroom, and kitchen styles.\n",
        "value" : "Provides information about property value (total value, gross tax, etc.)\n",
    }

In [16]:
def main():
    # Ask user for which table they would like to retrieve information from
    tables = get_tables()
    table_descriptions = get_table_descriptions()
    
# Welcome message and table selection
    print("Welcome to the Boston Property Assessment")
    print("Please select a table to explore properties:\n")
    for idx, table in enumerate(tables, start=1):
        description = table_descriptions.get(table, "No description available.")
        print(f"{idx}. {table} - {description}")
    table_choice = int(input("\nPlease select a table by typing in the corresponding number: ")) - 1
    selected_table = tables[table_choice]
    print(f"\nYou have selected the table: {selected_table}\n")

 # Ask user which columns they would like to view
    columns = get_columns(selected_table)
    print("What information would you like to retrieve from this table?\nHere are the available columns:")
    for idx, column in enumerate(columns, start=1):
        print(f"{idx}. {column}")
    column_choices = input("\nPlease enter the numbers of the columns you want to select (comma-separated): ")
    selected_columns = [columns[int(i) - 1] for i in column_choices.split(',')]
    selected_columns_str = ", ".join(selected_columns)
    print(f"\nYou selected columns: {selected_columns_str}\n")
    
    
 # This is a WHERE statement, allows users filter results comparative to a value (optional)
    where_clause = None
    filter_choice = input("Would you like to filter results based on a value (<, >, =)? \n Please type yes/no: ").strip().lower()
    if filter_choice == "yes":
        print("\nAvailable columns for filtering:")
        for idx, column in enumerate(columns, start=1):
            print(f"{idx}. {column}")
        where_col_idx = int(input("Enter the number of the column to filter by: ")) - 1
        where_column = columns[where_col_idx]
        operator = input("Choose an operator (=, <, >, <=, >=, LIKE): ").strip()
        value = input(f"Enter the comparative value for {where_column}: ").strip()
        value = f"'%{value}%'" if operator.upper() == "LIKE" else f"'{value}'" if not value.isdigit() else value
        where_clause = f"{where_column} {operator} {value}"
  

 # This is a JOIN statement, allows users to match with data from another table (optional)
    join_clause = None
    additional_columns = ""
    join_answer = input("\nWould you like to retrieve matching data from another table? Please type yes/no: ").strip().lower()
    if join_answer == "yes":
        print("\nAvailable tables:")
        for idx, table in enumerate(tables, start=1):
            print(f"{idx}. {table} - {table_descriptions.get(table, 'No description available.')}")
        join_table_choice = int(input("\nSelect a table to join by number: ")) - 1
        join_table = tables[join_table_choice]
        join_columns = get_columns(join_table)
        
        join_clause = f"JOIN {join_table} ON {selected_table}.p_id = {join_table}.p_id"
        
        # Ask the user if they want to display columns from the joined table 
        display_join_columns = input("\nWould you like to display columns from the joined table? (yes/no): ").strip().lower()
        if display_join_columns == "yes":
            print("\nAvailable columns in the joined table:")
            for idx, column in enumerate(join_columns, start=1):
                print(f"{idx}. {column}")
            join_column_choices = input("\nEnter the numbers of the columns to display from the joined table (comma-separated): ")
            selected_join_columns = [f"{join_table}.{join_columns[int(i) - 1]}" for i in join_column_choices.split(',')]
            additional_columns = ", " + ", ".join(selected_join_columns)
    
 # This is an ORDER BY statement, allows users to order their results in ascending or descending (optional)
    order_by = None
    order_by_choice = input("\nWould you like to sort the results? (yes/no): ").strip().lower()
    if order_by_choice == "yes":
        print("\nAvailable columns for sorting:")
        for idx, column in enumerate(columns, start=1):
            print(f"{idx}. {column}")
        order_col_idx = int(input("Enter the number of the column to sort by: ")) - 1
        order_column = columns[order_col_idx]
        order_direction = input("Choose sorting direction (1 for ASC, 2 for DESC): ").strip()
        order_by = f"{order_column} {'ASC' if order_direction == '1' else 'DESC'}"
    
 # This is a LIMIT statement, allows users to decide how many rows to display (optional)
    limit = input("Enter a LIMIT (default is 10): ").strip() or "10"
    

 # Building the query
    query = f"SELECT {selected_columns_str}{additional_columns} FROM {selected_table}"
    if join_clause:
        query += f" {join_clause}"
    if where_clause:
        query += f" WHERE {where_clause}"
    if order_by:
        query += f" ORDER BY {order_by}"
    query += f" LIMIT {limit}"
    
    print(f"\nHere is your query: {query}\n")
    
    
 # Execution and retrieval
    try:
        connection = get_db_connection()
        cursor = connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        
        if results:
            for row in results:
                print(row)
        else:
            print("No results found.")
        
        cursor.close()
        connection.close()
    except mysql.connector.Error as err:
        print(f"Error: {err}")

if __name__ == "__main__":
    main()

Welcome to the Boston Property Assessment
Please select a table to explore properties:

1. amenities - Provides a list of amenities for each property, such as heat, ac, parking, etc.

2. bldg_composite - Provides information about building structure class, roof, exterior, etc.

3. bldg_condition - Provide information about building condition (interior/exterior)

4. bldg_info - Provides information about building units (residential, commercial, etc)

5. mail - Provides information of a mailing address.

6. property - Provides information on property such as owner, street address, etc.

7. property_sf - Provides information about property space (land square footage, gross area, etc).

8. property_type - Provides information about number of buildings, land usage, etc.
 
9. rooms - Provides information about number of bedrooms, bathrooms, total rooms, etc.

10. style - Provides information about bedroom, bathroom, and kitchen styles.

11. value - Provides information about property value (

KeyboardInterrupt: Interrupted by user

## This is a function that allows users to add or remove values from the table
In the ideal environment, we would only allow authorized users to add or remove values from the database.

In [20]:
def add_property():
    
    """Function to add a new property and related data.
        Users are first asked to input data for the property table, then asked if they have additional 
        information to add onto matching tables. Then using the p_id adding a new row to the table. """
    try:
        connection = get_db_connection()
        cursor = connection.cursor()

        # Asking user to input data for the Property Table. 
        # This information is CRUCIAL to our database, so users can only add a property if they are able to fill out the table. 
        print("\n\n You have selected: Adding a new property to the database.")
        print("\n Please note, in order to add a property you must input property_id, owner, and complete property address")
        print("Please provide the following details:")

        # Gathering user input
        p_id = input("Property ID (p_id a 10 digit unique ID): ")
        owner = input("Owner: ")
        st_num = input("Street Number (st_num): ")
        st_name = input("Street Name (st_name): ")
        unit_num = input("Unit Number (unit_num): ")
        city = input("City: ")
        zipcode = input("Zipcode: ")

        # Inserting data into property table
        add_property_query = """
            INSERT INTO property (p_id, owner, st_num, st_name, unit_num, city, zipcode)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(add_property_query, (p_id, owner, st_num, st_name, unit_num, city, zipcode))
        connection.commit()

        print(f"\nNew property added with ID: {p_id}")

        # Ask user if they would like to add more information to other tables, matching to the p_id value.
        # This statement runs until the user has no more information to add to the table.
        while True:
            add_related_data = input("\nWould you like to add matching information to another table? (yes/no): ").strip().lower()
            if add_related_data != "yes":
                break

            tables = get_tables()
            print("\nAvailable tables to add data to:")
            for idx, table in enumerate(tables, start=1):
                if table != "property":  # Skip the property table
                    print(f"{idx}. {table}")

            table_choice = input("\nPlease enter only the number of the table you'd like to add data to (or press Enter to skip): ").strip()
            if not table_choice:
                print("No table selected. Exiting data addition.")
                break

            selected_table = tables[int(table_choice) - 1]

            # Retriving available columns from the table
            columns = get_columns(selected_table)
            print(f"\nAvailable columns in {selected_table}:")
            for idx, column in enumerate(columns, start=1):
                print(f"{idx}. {column}")

            # Ask the user which columns they want to input data for
            column_choices = input(
                "Enter the numbers of the columns you want to add values for (comma-separated, or press Enter to skip): "
            ).strip()

            # We will be matching using the p_id
            if column_choices:
                selected_columns = [columns[int(i) - 1] for i in column_choices.split(',')]
                column_data = {}
                for column in selected_columns:
                    if column == "p_id":  
                        column_data[column] = p_id
                    else:
                        column_data[column] = input(f"Enter value for {column}: ")

                # The INSERT query for the table. 
                columns_str = ", ".join(column_data.keys())
                placeholders = ", ".join(["%s"] * len(column_data))
                insert_query = f"INSERT INTO {selected_table} ({columns_str}) VALUES ({placeholders})"
                cursor.execute(insert_query, tuple(column_data.values()))
                connection.commit()

                print(f"Data added to {selected_table} was successful!")
            else:
                print(f"Skipping table: {selected_table}")

        print("\n Data addition tasks completed, database successfully updated!")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        connection.close()

In [21]:
def remove_property():
    """Function to remove a property and its related data.
        
        The p_id is a unique key for each table. 
        This function only asks for the p_id for the table, then it deletes the matching p_id for all other tables
        that have matching p_id. """
    try:
        connection = get_db_connection()
        cursor = connection.cursor()

        # Ask user for the p_ID to delete
        p_id = input("\nEnter the Property ID (p_id) of the property you want to remove: ").strip()

        # Check if the p_id exists
        cursor.execute("SELECT * FROM property WHERE p_id = %s", (p_id,))
        property_exists = cursor.fetchone()

        if not property_exists:
            print(f"\nProperty ID {p_id} does not exist in the database.")
            return

        # Confirm deletion
        confirm = input(f"Are you sure you want to delete property {p_id} and all related data? (yes/no): ").strip().lower()
        if confirm == "yes":
            # Delete related data from all tables manually (if no ON DELETE CASCADE is set up)
            tables = get_tables()
            for table in tables:
                delete_query = f"DELETE FROM {table} WHERE p_id = %s"
                cursor.execute(delete_query, (p_id,))
            connection.commit()

        # Confirm to user that action was taken
            print(f"\nProperty {p_id} and all related data have been removed from the database.")
        else:
            print("\nTask was not executed.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        connection.close()

In [22]:
def main_menu():
    """Main menu to choose between adding or removing data.
        This function asks the user what they would like to accomplish, 
        then running corresponding function above."""
    
    while True:
        print("\nWelcome authorized editor of the Property Database.")
        print("Please select an action to modify the database: (Enter the corresponding number..)")
        print("1. Add a new property")
        print("2. Remove a property")
        print("3. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            add_property()
        elif choice == "2":
            remove_property()
        elif choice == "3":
            print("Exiting...")
            break
        else:
            print("Please try again.")


if __name__ == "__main__":
    main_menu()


Welcome authorized editor of the Property Database.
Please select an action to modify the database: (Enter the corresponding number..)
1. Add a new property
2. Remove a property
3. Exit
Enter your choice: 3
Exiting...
